In [1]:
import os
import requests
import time
from PIL import Image
from io import BytesIO
from geopy.geocoders import Nominatim
import json

# Функция для загрузки изображения и получения метаданных
def download_image(api_key, location, pitch, heading):
    params = {
        'size': '640x640',
        'location': location,
        'pitch': pitch,
        'heading': heading,
        'fov': 90,
        'key': api_key
    }
    response = requests.get('https://maps.googleapis.com/maps/api/streetview', params=params)
    img = Image.open(BytesIO(response.content))
    metadata = json.loads(response.headers['X-Goog-Streetview-Metadata'])
    return img, metadata

# Настройки
api_key = 'ВАШ API-КЛЮЧ GOOGLE STREET VIEW'
district = 'Москва'
output_dir = 'dataset'
num_images_per_building = 5
min_floors = 3
max_floors = 30

# Определение границ района
geolocator = Nominatim(user_agent='myapplication')
location = geolocator.geocode(district)
bounds = location.raw['boundingbox']

# Создание директории для сохранения изображений
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Сбор координат для зданий в определенном районе
coords = []
for i in range(int(bounds[0]), int(bounds[1])+1):
    for j in range(int(bounds[2]), int(bounds[3])+1):
        address = geolocator.reverse(f'{i}, {j}').address
        if 'улица' in address or 'проспект' in address or 'переулок' in address:
            coords.append((i, j))

# Загрузка изображений и сохранение метаданных для зданий
count = 0
for coord in coords:
    for i in range(num_images_per_building):
        try:
            img, metadata = download_image(api_key, f'{coord[0]},{coord[1]}', 0, i*60)
            num_floors = metadata['floors']
            if num_floors >= min_floors and num_floors <= max_floors:
                img.save(f'{output_dir}/image_{count}.jpg')
                with open(f'{output_dir}/metadata_{count}.json', 'w') as f:


5
